In [22]:
 # Dependencies and Setup
import pandas as pd

In [23]:
# Files to Load
student_input = "Resources/students_complete.csv"
school_input = "Resources/schools_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
student_data = pd.read_csv(student_input)
school_data = pd.read_csv(school_input)

# Combine the data into a single dataset.  
school_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [24]:
school_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

**Create a high-level snapshot, in a DataFrame, of the district's key metrics, including the following:**

- Total schools
- Total students
- Total budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

In [26]:
# district_df = school_df[[]]

school_df["school_name"].nunique()
school_df["school_name"].unique()


array(['Huang High School', 'Figueroa High School', 'Shelton High School',
       'Hernandez High School', 'Griffin High School',
       'Wilson High School', 'Cabrera High School', 'Bailey High School',
       'Holden High School', 'Pena High School', 'Wright High School',
       'Rodriguez High School', 'Johnson High School', 'Ford High School',
       'Thomas High School'], dtype=object)

In [27]:
school_df.describe(include='all')

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
count,39170.000000,39170,39170,39170,39170,39170.00000,39170.000000,39170.000000,39170,39170.000000,3.917000e+04
unique,NaN,32715,2,4,15,NaN,NaN,NaN,2,NaN,NaN
top,NaN,Michael Smith,F,9th,Bailey High School,NaN,NaN,NaN,District,NaN,NaN
freq,NaN,26,19735,11408,4976,NaN,NaN,NaN,26976,NaN,NaN
mean,19584.500000,NaN,NaN,NaN,NaN,81.87784,78.985371,6.978172,NaN,3332.957110,2.117241e+06
std,11307.549359,NaN,NaN,NaN,NaN,10.23958,12.309968,4.444329,NaN,1323.914069,8.749987e+05
min,0.000000,NaN,NaN,NaN,NaN,63.00000,55.000000,0.000000,NaN,427.000000,2.480870e+05
25%,9792.250000,NaN,NaN,NaN,NaN,73.00000,69.000000,3.000000,NaN,1858.000000,1.081356e+06
50%,19584.500000,NaN,NaN,NaN,NaN,82.00000,79.000000,7.000000,NaN,2949.000000,1.910635e+06
75%,29376.750000,NaN,NaN,NaN,NaN,91.00000,89.000000,11.000000,NaN,4635.000000,3.022020e+06


## School Summary

**Create a DataFrame that summarizes key metrics about each school, including the following:**

- School name
- School type
- Total students
- Total school budget
- Per student budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

## Highest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the top 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

## Lowest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the bottom 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

## Math Scores by Grade

Create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

## Reading Scores by Grade

Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

## Scores by School Spending

Create a table that breaks down school performance based on average spending ranges (per student). Use your judgment to create four bins with reasonable cutoff values to group school spending. Include the following metrics in the table:

* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

## Scores by School Size

Create a table that breaks down school performance based on school size (small, medium, or large).

## Scores by School Type

Create a table that breaks down school performance based on type of school (district or charter).